# Random Forests

## What are they?

From the words of Jeremy Howard himself (from the YouTube transcript of the lectures):

> in brief a random forest is a
kind of universal machine learning
technique it's a way of predicting
something that can be of any kind it
could be a category like is it a dog or
a cat or it could be a continuous
function you aspera like price it can
predict it with columns of pretty much
any kind pixel data zip codes revenues
whatever in general it doesn't overfit
it can and will learn to check whether
it is but it doesn't generally overfit
too badly and it's very very easy to
make to stop it from overfitting you
don't need and we'll talk more about
this you don't need a separate
validation set in general it can tell
you how well it
generalizes even if you only have one
data set it has few if any statistical
assumptions it doesn't assume that your
data is normally distributed it doesn't
assume that the relationship so linear
it doesn't assume that you've just
specified the interactions it requires
very few pieces of feature engineering
for many different types of situation
you don't have to take the log of the
data you don't have to multiply
interactions together so in other words
it's a great place to start right if
your first random forest does very
little useful then that's a sign that
there might be problems with your data
like it's designed to work pretty much
first off >

Summarizing, it's a universal machine learning technique. Why?

1. It's a way of predicting either a category or regressor. 
1. It can handel any type of columns (either continuous or not). It can handle them as categorical variables (ordinal or not) or one-hot encoding. This last part is due to the fact that it can split multiple times and arrive at a split that is essentially a dummy for any of the categories. 
1. In general, it does not overfit that easily and has very few parameters to check if indeed it does overfit. 
1. You can use Out Of Bags samples as validation samples. 
1. It has few if any statistcial assumptions. i.e., it does not assume a linear relationship nor does assume which interactions to use. Thus, it requires very few pieces of feature engineering. For clarification of how big this is, compare it to a logistic regression: 
> it's pretty much always possible to
create a simple like logistic regression
which is as good as pretty much any
random first if you know ahead of time
exactly what variables you need exactly
how they interact exactly how they need
to be transformed and so forth right so >

Compare this to the F. Chollet's remarks about the multi-stage nature of Neural Networks and the resulting data-based feature engineering. 

6. It scales very well as the number of observations grow. This is a result from the bootstraping used to decorrelate the weak learners: i.e., you can train each tree on a sample from the whole dataset. 

## How does it work?

A forest is a set of trees. Thus, let's study trees first:

### Decision Trees

A decision tree is a machine learning algorithm that works by *iteratively* and *greedily* split the decision space into regions. Each element to predict is then placed on one of the regions and predicted with the mean of the elements in the region (or majority class in it). To do so, it works like thus:

In [ ]:
def find_best_split():
    # Check all the possible splits for a given var
    # Return the split that minimizes the error. 

def find_one_split():
    # Find for each var the best split possible (`find_best_split`)
    # Use the one that yields the lowest error

while (no_stopping_rule):
    find_one_split()

Thus, the algorithm iteratively splits and splits until a stopping rule is given. That stopping rule can be:

- The max_depth of the tree. i.e., how many splits to perform.
- The min_num_in_leaf. i.e., the minimum number of observations to have at the end of the tree at each region. 

## Bagging 

Bagging is a machine learning technique for combining multiple *uncorrelated weak learners* and averaging over them to perform the final prediction. Weak learners because, by themselves, none of them performs particurly well: that is, all of them commit prediction errors. However, because some trees overpredict and others underpredict (that is, they commit different errors), when averaging the errors cancel each other and the resulting prediction is much better than the resulting from any individual tree. 

Thus, the way to create a forest using bagging is to ensure that the trees are uncorrelated. Random forest uses two techniques to decorrelate them:

1. Bootstraped samples for each tree to work all the way down. 
2. Random subset of features at each split for each tree.

### Bootstraped samples

Given that each tree is possibly working with a very different sample from the dataset, the resulting predictions will be uncorrelated. Thus, as we average over this different trees, our resulting prediction will be better. 

#### Out of Bag (OOB) Samples

Given the nature of our experiment, each observation won't be used (possibly) in some of the trees. Thus, using those trees, we could predict that observation and use it as a validation proxy. Note that because we did not use all the trees in the prediction (the trees that did use that validation in their training), **our prediction will not be as good as it could possibly be**. 

If we repeat this process for each observation in the dataset, we would have done it for what amounts as a validation set. Thus, this out of bag samples are a type of validation set. Note that because we are not using our best predictions to do so, ** this out of bag (OOB) error should underperform relative to a proper validation set error**. 

### Random subset of features

We may use bootstraped samples, and still not get uncorrelated (different) predictions if, no matter what subsample each tree uses, the same features are always used. To avoid a very predictive set of features to dominate each weak learner, we can do the following: **at each possible split at each tree, consider only a random subset of the features to find the best possible split**. Thus, predictions at each tree will be ever more uncorrelated and thus the averaging will work even better. 

## Python code

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))
    
def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))
    
def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

To modify how many samples are passed to each tree, and thus speed up or decrease the training process and thus do it more iteratively. Even, to use random forest on large samples as we said on the introduction. 

In [ ]:
set_rf_samples(1000) # give each tree just 1000 samples
reset_rf_samples() # Undoes the changes produced by set_rf_samples.

### Hyperparameters

In [ ]:
RandomForestRegressor(n_estimators=40, min_samples_leaf=3, 
                      max_features=0.5, n_jobs=-1, oob_score=True)
# Hyperparameters to tune with scikit-learn

## Model-Driven Exploratory Data Analysis

Jeremy says that you should get a Random Forest up and running as fast as possible, not only because it simply works, but because it will allow you to do **model-driven exploratory data analysis**. Given the algorithm's flexibility, you can thus do the following: 

- confidence on predictions. 
- feature importance. 
- partial dependence. 
- tree interpreter. 
- see which features extrapolate well to the validation/test set (using feature importance).

Note that using the technique that Jeremy uses to compute feature importance (training and introducing noise in a variable and see how does the error change), we can also do the same for any machine learning algortihm. 

### Confidence on predictions

Due to the nature of bagging, each observation is predicted using multiple single trees. To see how confident we are on each prediction, we can compute the standard deviation of these predictions.

### Feature importance

We can train the model as we regularly do and then do the following for each variable:

1. Randomly shuffle the values for that variable across the dataset, thus making the variable pure noise.
2. Predict this altered dataset and see how did the training error changed. 
3. Annotate this change in the training error. 

Thus, the most important features will be those that caused the worst change in the training error. 

Note: scikit-learn can use a different way (ensemble-tree specific) to calculate feature importance. 

### Partial dependence

Sometimes we wonder, what would happen were we only change one variable and let all the others stay the same. We can train, change the all of the values for a given variable to a specific one and plot the result for each observation. Then do the same for all the unique values in the given variable and join the dots for each observation. Doing so, we have a partial plot dependence that allows us to examine what happens for each observation as we change the values of only one variable. 

### Tree interpreter

Given a prediction, we can calculate how much of the change, relative to the dataset average, is due to each of the variables. To do so, we traverse each tree and check every split: annotate the variable that gave the split, and annotate the change in mean prediction relative to the last region. We do so for each tree and at the end we will have annotated how much the mean changed due to each of the variables used in the overall algorithm.

### Extrapolation

Extrapolation is a very cool trick when you have non-random validation/test sets. First thing to do is see if the validation is non-random (this is obvious when you are working with a date component) by predicting whether a variable is or is not in the validation/test set using all of your variables. If you get a good prediction, then the validation/test set is not random.

You could do feature importance on this model and thus know which of the variables are the ones that change the most from the training to the validation/test set. Then, you could start dropping some of them as they are essentially noise and thus improve your overall fit on the validation set. That is, you focus on the variables that essentially work. 